In [23]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)
pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [24]:
bnf = pd.read_csv('BNF_mtrend.csv')
nifty = pd.read_csv('NIFTY_mtrend.csv')
# fnf = pd.read_csv('fnf_mtrend.csv')
midcp = pd.read_csv('MIDCP_mtrend.csv')
sensex = pd.read_csv('SENSEX_mtrend.csv')

bnf['Index'] = 'BANKNIFTY'
nifty['Index'] = 'NIFTY'
# fnf['Index'] = 'FINNIFTY'
midcp['Index'] = 'MIDCPNIFTY'
sensex['Index'] = 'SENSEX'

In [25]:
combined_df = pd.concat([bnf, nifty, midcp, sensex], ignore_index=True)
# combined_df.drop(columns=['Unnamed: 0'], inplace=True)

In [26]:
combined_df['Signal Generated At'] = pd.to_datetime(combined_df['Signal Generated At'])
combined_df = combined_df.sort_values(by='Signal Generated At')

In [27]:
# combined_df['Index']

In [28]:
# bnf = bnf.dropna(subset=['Signal Generated At'])
# nifty = nifty.dropna(subset=['Signal Generated At'])
# fnf = fnf.dropna(subset=['Signal Generated At'])

In [29]:
combined_df['Allocation'] = 6250000
combined_df.loc[((combined_df['Trade Year'] >= 2017) & (combined_df['Trade Year'] <= 2021), 'Portfolio')] = 18750000
combined_df.loc[((combined_df['Trade Year'] >= 2022) & (combined_df['Trade Year'] <= 2025), 'Portfolio')] = 25000000
# combined_df.loc[((combined_df['Trade Year'] == 2022) & (combined_df['Trade Month'] >= 10)) | ((combined_df['Trade Year'] == 2023) & (combined_df['Trade Month'] <= 8)), 'Allocation'] = 6666666
# combined_df.loc[((combined_df['Trade Year'] >= 2023) & (combined_df['Trade Month'] >= 9)) | (combined_df['Trade Year'] >= 2024), 'Allocation'] = 5000000

In [30]:
# combined_df['Leverage'] = combined_df['Index'].map({
#     'BANKNIFTY': 5,
#     'FINNIFTY': 15
# }).fillna(8)  # Default leverage is 8 if neither BANKNIFTY nor FINNIFTY

# Continue with calculations
# combined_df['Qty'] = combined_df['Allocation'] * combined_df['Leverage'] / combined_df['Strike']
# combined_df['PnL'] = combined_df['After Costs'] * combined_df['Qty']
# combined_df['ROI%'] = combined_df['PnL'] * 100 / combined_df['Allocation']
# combined_df['ROI% PF'] = combined_df['PnL'] * 100 / combined_df['Portfolio']

In [31]:
combined_df.tail()

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Allocation,Portfolio
1288,2025-02-18 12:15:00,LONG,2025-02-18 13:15:00,11058.7000,11001.0000,11001.0000,2025-02-19 09:15:00,11001.0000,-57.7000,-68.7299,-247427.4600,Initial SL Hit,3600,6.3330,-3.9588,2025,2,MIDCPNIFTY,6250000,25000000.0000
1142,2025-02-18 13:15:00,LONG,2025-02-18 14:15:00,22934.3000,22871.7500,22871.7500,2025-02-19 09:15:00,22847.5500,-86.7500,-95.9064,-165438.4882,Gap Open Outside ISL,1725,6.3330,-2.6470,2025,2,NIFTY,6250000,25000000.0000
1289,2025-02-19 13:15:00,LONG,2025-02-19 14:15:00,11261.3500,11212.0000,11212.0000,2025-02-19 14:15:00,11212.0000,-49.3500,-60.5867,-210841.6290,Initial SL Hit,3480,6.3330,-3.3735,2025,2,MIDCPNIFTY,6250000,25000000.0000
1290,2025-02-19 15:15:00,LONG,2025-02-20 09:15:00,11282.3000,11255.0500,11367.0978,2025-02-21 09:15:00,11255.0500,-27.2500,-38.5187,-134044.9890,Initial SL hit,3480,6.3330,-2.1447,2025,2,MIDCPNIFTY,6250000,25000000.0000
1143,2025-02-20 15:15:00,LONG,2025-02-21 09:15:00,22920.8000,22905.0000,22905.0000,2025-02-21 09:15:00,22905.0000,-15.8000,-24.9652,-43064.9010,Initial SL hit,1725,6.3330,-0.6890,2025,2,NIFTY,6250000,25000000.0000


In [32]:
combined_df['ROI% PF'] = combined_df['PnL'] * 100 / combined_df['Portfolio']

In [33]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            
            "Max Drawdown",
            "ROI/DD Ratio",
            
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return pd.DataFrame(stats_df8)

In [34]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,44.8983,263,27.7567,1.9942,-0.5299,-16.8076,2.6713
2018,70.4373,185,30.8108,2.6242,-0.6183,-20.5450,3.4284
2019,70.9086,196,29.0816,2.8249,-0.6483,-16.7585,4.2312
2020,250.9313,171,35.0877,5.9691,-0.9659,-20.3011,12.3605
2021,107.8662,191,28.7958,4.0107,-0.8288,-20.8301,5.1784
2022,89.2975,232,25.4310,3.5410,-0.6915,-23.0153,3.8799
2023,121.3847,236,33.0508,2.4394,-0.4360,-18.0120,6.7391
2024,45.5946,250,26.0000,2.3672,-0.5853,-22.0086,2.0717
2025,-20.6270,45,11.1111,1.8916,-0.7521,-20.2179,-1.0202
Overall,780.6916,1769.0000,28.7733,3.1384,-0.6482,-29.5687,26.4026


# Mtrend - BNF, MIDCP, NIFTY, SENSEX

In [43]:
combined_df["DD%"] = combined_df["ROI% PF"].cumsum() - combined_df["ROI% PF"].cumsum().cummax()

In [44]:
combined_df.tail(12)

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Allocation,Portfolio,ROI% PF,DD%,Time In Trade,Premium Cost,Final Points,Final ROI%
1766,2025-02-10 10:15:00,LONG,2025-02-10 11:15:00,77365.9100,77189.0400,77189.0400,2025-02-10 11:15:00,77189.0400,-176.8700,-192.3255,-100009.2574,Initial SL Hit,520,6.3330,-1.6001,2025,2,SENSEX,6250000,25000000.0000,-0.4269,-25.6972,1,12.8943,-205.2198,-0.4269
690,2025-02-10 10:15:00,LONG,2025-02-10 11:15:00,49867.0500,49703.1000,49703.1000,2025-02-11 09:15:00,49703.1000,-163.9500,-173.9070,-109561.4195,Initial SL Hit,630,5.0660,-1.7530,2025,2,BANKNIFTY,6250000,25000000.0000,-0.4592,-26.1563,1,8.3112,-182.2182,-0.4592
1767,2025-02-10 12:15:00,LONG,2025-02-10 13:15:00,77244.6900,77158.0600,77158.0600,2025-02-10 13:15:00,77158.0600,-86.6300,-102.0703,-53076.5430,Initial SL Hit,520,6.3330,-0.8492,2025,2,SENSEX,6250000,25000000.0000,-0.2391,-26.3954,1,12.8741,-114.9444,-0.2391
1768,2025-02-11 10:15:00,LONG,2025-02-11 11:15:00,77037.4400,76908.8100,76908.8100,2025-02-11 11:15:00,76908.8100,-128.6300,-144.0246,-74892.8050,Initial SL Hit,520,6.3330,-1.1983,2025,2,SENSEX,6250000,25000000.0000,-0.3263,-26.7217,1,12.8396,-156.8642,-0.3263
691,2025-02-11 15:15:00,LONG,2025-02-12 09:15:00,49436.1000,49344.4000,49344.4000,2025-02-12 09:15:00,49344.4000,-91.7000,-101.5780,-63994.1715,Initial SL Hit,630,5.0660,-1.0239,2025,2,BANKNIFTY,6250000,25000000.0000,-0.2767,-26.9984,1,8.2393,-109.8174,-0.2767
692,2025-02-12 11:15:00,LONG,2025-02-12 12:15:00,49449.6000,48975.1000,49428.3556,2025-02-13 13:15:00,49349.0000,-100.6000,-110.4799,-69602.3118,TSL Hit,630,5.0660,-1.1136,2025,2,BANKNIFTY,6250000,25000000.0000,-0.3199,-27.3184,2,16.4832,-126.9631,-0.3199
1287,2025-02-17 13:15:00,LONG,2025-02-17 14:15:00,11177.9000,11049.2500,11049.2500,2025-02-18 10:15:00,11049.2500,-128.6500,-139.7636,-503148.8700,Initial SL Hit,3600,6.3330,-8.0504,2025,2,MIDCPNIFTY,6250000,25000000.0000,-2.0394,-29.3578,1,1.8630,-141.6266,-2.0394
1288,2025-02-18 12:15:00,LONG,2025-02-18 13:15:00,11058.7000,11001.0000,11001.0000,2025-02-19 09:15:00,11001.0000,-57.7000,-68.7299,-247427.4600,Initial SL Hit,3600,6.3330,-3.9588,2025,2,MIDCPNIFTY,6250000,25000000.0000,-1.0163,-30.3741,1,1.8431,-70.5730,-1.0163
1142,2025-02-18 13:15:00,LONG,2025-02-18 14:15:00,22934.3000,22871.7500,22871.7500,2025-02-19 09:15:00,22847.5500,-86.7500,-95.9064,-165438.4882,Gap Open Outside ISL,1725,6.3330,-2.6470,2025,2,NIFTY,6250000,25000000.0000,-0.6881,-31.0622,1,3.8224,-99.7288,-0.6881
1289,2025-02-19 13:15:00,LONG,2025-02-19 14:15:00,11261.3500,11212.0000,11212.0000,2025-02-19 14:15:00,11212.0000,-49.3500,-60.5867,-210841.6290,Initial SL Hit,3480,6.3330,-3.3735,2025,2,MIDCPNIFTY,6250000,25000000.0000,-0.8695,-31.9317,1,1.8769,-62.4636,-0.8695


In [37]:
#### combined_df.to_csv('MTrend_nbms_1x.csv', index=False)

# MTrend - BNF, NF, MIDCP, SENSEX

In [38]:
combined_df['Entry Time'] = pd.to_datetime(combined_df['Entry Time'])
combined_df['Exit Time'] = pd.to_datetime(combined_df['Exit Time'])
combined_df['Time In Trade'] = (combined_df['Exit Time'] - combined_df['Entry Time'] + dt.timedelta(days=1)).dt.days

In [39]:
combined_df['Premium Cost'] = (combined_df['Time In Trade'] * 0.5/30) * combined_df['Entry Price'] / 100

In [40]:
combined_df['Final Points'] = combined_df['After Costs'] - combined_df['Premium Cost']
combined_df['Final ROI%'] = combined_df['Final Points'] * combined_df['Qty'] * 100 / combined_df['Portfolio']

In [41]:
combined_df['PnL'].sum() , combined_df['Final Points'].sum()

(161107780.37126, 85039.11836766657)

In [42]:
combined_df['ROI% PF'] = combined_df['Final ROI%']
stats2 = generate_stats(combined_df)
stats2

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,19.1121,263,25.0951,1.9530,-0.5573,-19.8073,0.9649
2018,48.6179,185,27.5676,2.6335,-0.6395,-22.3461,2.1757
2019,50.1950,196,24.4898,3.0887,-0.6626,-19.7637,2.5397
2020,227.9310,171,34.5029,5.7790,-1.0092,-20.9079,10.9017
2021,85.6465,191,27.7487,3.8947,-0.8752,-22.9078,3.7387
2022,67.7145,232,24.5690,3.4061,-0.7225,-24.3359,2.7825
2023,96.4898,236,31.3559,2.3273,-0.4675,-21.6532,4.4562
2024,22.4233,250,26.0000,2.1285,-0.6266,-24.1461,0.9287
2025,-23.0089,45,11.1111,1.7300,-0.7915,-22.5783,-1.0191
Overall,595.1211,1769.0000,27.0209,3.0799,-0.6794,-32.7188,18.1889


# MTrend -> 4 indices after Premium Costs

In [13]:
# combined_df.to_csv('MTrend tb.csv', index=False)

In [86]:
# opt_buy = pd.read_csv('BNF_FNF_NF_MIDCP_Combined_MTrend_Opt_Buying.csv')
# combined_df = pd.concat([bnf, fnf, nifty, midcp], ignore_index=True)
# combined_df.drop(columns=['Unnamed: 0'], inplace=True)
# combined_df['Entry Time'] = pd.to_datetime(combined_df['Entry Time'])
# combined_df = combined_df.sort_values(by='Entry Time')

In [87]:
# combined_df

In [88]:
# combined_df = combined_df.drop(columns=['DATETIME', 'TODAYS OPEN', 'TODAYS CLOSE', 'POINTS CAPTURED TODAY', 'QTY', 'MTM', 'DD%'])
# combined_df['PnL'] = np.where(combined_df['Strategy'].isna(), 'OPT BUY', 'POSITIONAL')

In [89]:
# combined_df2.to_csv('Combined MTrend Shotgun.csv')

In [90]:
# combined_df2 = pd.concat([combined_df, opt_buy], ignore_index=True)
# combined_df2.drop(columns=['Unnamed: 0'], inplace=True)
# combined_df2['Entry Time'] = pd.to_datetime(combined_df2['Entry Time'])
# combined_df2 = combined_df2.sort_values(by='Entry Time')
# combined_df2 = combined_df2.drop(columns=['DATETIME', 'TODAYS OPEN', 'TODAYS CLOSE', 'POINTS CAPTURED TODAY', 'QTY', 'MTM', 'DD%'])
# combined_df2['Strategy'] = np.where(combined_df2['THEO PnL'].isna(), 'OPT BUY', 'POSITIONAL')

In [91]:
# combined_df2

In [92]:
# combined_df2 = combined_df2.drop(columns=['Drawdown', 'Cumulative ROI%', 'Running Max ROI%'])

In [93]:
# combined_df2['entry_date'] = np.where(combined_df2['Expiry'].isna(), combined_df2['Entry Time'].dt.date, combined_df2['Expiry'])

In [94]:
def generate_stats(tb):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
        ],
    )
    combined_df_sorted = tb
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
    ]
    return stats_df8

In [95]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,68.5217,233,28.7554,2.4661,-0.5826,-15.8957,4.3107
2018,62.1041,184,29.3478,2.8728,-0.7156,-24.0541,2.5818
2019,92.7731,176,25.5682,3.9985,-0.6653,-19.7932,4.6871
2020,239.1324,166,32.5301,6.5561,-1.0259,-22.2313,10.7566
2021,95.6872,176,27.8409,4.2385,-0.8819,-18.4246,5.1935
2022,94.0058,226,24.7788,3.8474,-0.7144,-20.0542,4.6876
2023,109.1013,234,32.0513,2.4398,-0.4647,-18.4958,5.8987
2024,44.8799,179,26.8156,2.5621,-0.5962,-20.5050,2.1887
Overall,806.2054,1574.0000,28.4625,3.5344,-0.6902,-24.0541,33.5163


# MTrend NSE Indices New

In [96]:
combined_df.to_csv('MTrend NSE Indices New.csv', index=False)